In [ ]:
import socket
import struct

# -----------------------------
# 1) UDP 监听配置
# -----------------------------
UDP_IP   = "0.0.0.0"   # 本机监听地址(所有网卡)
UDP_PORT = 10087

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

# -----------------------------
# 2) 数据格式
# -----------------------------
# 发送端约定：77 个 double（小端序），共 616 字节
EXPECTED_LEN   = 77
EXPECTED_BYTES = EXPECTED_LEN * 8
fmt = "<" + "d" * EXPECTED_LEN  # 小端序 + 77 个 double

# 字段名（列名）与发送端顺序对应
col_names = [
    "Time",            # values[0]  = sim_time
    "y_spcktime",      # values[1]
    "y_cb_vx",         # values[2]
    "y_cb_x",          # values[3]
    "y_cb_y",          # values[4]
    "y_cb_z",          # values[5]
    "y_cb_roll",       # values[6]
    "y_cb_yaw",        # values[7]
    "y_cb_pitch",      # values[8]
    "y_w01_rotw",      # values[9]
    "y_w02_rotw",      # values[10]
    "y_w03_rotw",      # values[11]
    "y_w04_rotw",      # values[12]
    "y_w05_rotw",      # values[13]
    "y_w06_rotw",      # values[14]
    "y_w07_rotw",      # values[15]
    "y_w08_rotw",      # values[16],
    "y_f01_x",         # values[17]
    "y_f01_y",         # values[18]
    "y_f01_z",         # values[19]
    "y_f01_roll",      # values[20]
    "y_f01_yaw",       # values[21]
    "y_f01_pitch",     # values[22]
    "y_f02_x",         # values[23]
    "y_f02_y",         # values[24]
    "y_f02_z",         # values[25]
    "y_f02_roll",      # values[26]
    "y_f02_yaw",       # values[27]
    "y_f02_pitch",     # values[28]
    "y_ws01_x",        # values[29]
    "y_ws01_y",        # values[30]
    "y_ws01_z",        # values[31]
    "y_ws01_roll",     # values[32]
    "y_ws01_yaw",      # values[33]
    "y_ws01_pitch",    # values[34]
    "y_ws02_x",        # values[35]
    "y_ws02_y",        # values[36]
    "y_ws02_z",        # values[37]
    "y_ws02_roll",     # values[38]
    "y_ws02_yaw",      # values[39]
    "y_ws02_pitch",    # values[40]
    "y_ws03_x",        # values[41]
    "y_ws03_y",        # values[42]
    "y_ws03_z",        # values[43]
    "y_ws03_roll",     # values[44]
    "y_ws03_yaw",      # values[45]
    "y_ws03_pitch",    # values[46]
    "y_ws04_x",        # values[47]
    "y_ws04_y",        # values[48]
    "y_ws04_z",        # values[49]
    "y_ws04_roll",     # values[50]
    "y_ws04_yaw",      # values[51]
    "y_ws04_pitch",    # values[52]
    "y_w01_rota",      # values[53]
    "y_w02_rota",      # values[54]
    "y_w03_rota",      # values[55]
    "y_w04_rota",      # values[56]
    "y_w05_rota",      # values[57]
    "y_w06_rota",      # values[58]
    "y_w07_rota",      # values[59]
    "y_w08_rota",      # values[60]
    "y_bar01_pitch",   # values[61]
    "y_bar02_pitch",   # values[62]
    "y_bar03_pitch",   # values[63]
    "y_bar04_pitch",   # values[64]
    "y_bar05_pitch",   # values[65]
    "y_bar06_pitch",   # values[66]
    "y_bar07_pitch",   # values[67]
    "y_bar08_pitch",   # values[68]
    "y_ws01_vy",       # values[69]
    "y_ws02_vy",       # values[70]
    "y_ws03_vy",       # values[71]
    "y_ws04_vy",       # values[72]
    "y_ws01_vyaw",     # values[73]
    "y_ws02_vyaw",     # values[74]
    "y_ws03_vyaw",     # values[75]
    "y_ws04_vyaw",     # values[76]
]

# -----------------------------
# 3) 缓存与文件输出逻辑
# -----------------------------
all_data = []        # 用于缓存全部行
threshold = 49.99    # 当 Time(=sim_time) >= 49.99 时，写出并退出
got_threshold = False

print(f"Listening on {UDP_IP}:{UDP_PORT} ...")

while True:
    data, addr = sock.recvfrom(65535)
    if len(data) != EXPECTED_BYTES:
        print(f"[WARN] Received {len(data)} bytes, expected {EXPECTED_BYTES} -> skipping")
        continue

    # 解析出 77 个 double
    values = struct.unpack(fmt, data)

    # 缓存该帧
    all_data.append(values)

    # 检查 Time(=values[0]) 是否超过 49.99
    if values[0] >= threshold and not got_threshold:
        got_threshold = True
        print(f"[INFO] sim_time >= {threshold}. Writing results to 'Result_UDPY_RosRt.log'...")

        # ---------------------------
        # 4) 写文件
        # ---------------------------
        with open("Result_UDPY_RosRt.log", "w") as f:
            # 写表头(带引号，用\t分隔)
            header_line = "\t".join(f'"{col}"' for col in col_names)
            f.write(header_line + "\n")

            # 写每行数据(按顺序输出，与col_names对应)
            for row in all_data:
                # row 是一个包含 77 个 float/double 的 tuple
                # 这里统一格式化为 6 位小数，可自行调整
                row_str = "\t".join(f"{v:.6f}" for v in row)
                f.write(row_str + "\n")

        print("[INFO] Done. Exiting script.")
        break  # 退出脚本（若需继续监听可改为 pass 或设别的逻辑）
